In [2]:
import pandas as pd
import time
from collections import defaultdict
from itertools import product
import pickle

SHW = 100000

In [3]:
# g = pd.read_csv("data/attendances_over_time.csv.gz", encoding='latin-1', 
#                 header=None, names=["cust_id","age_gr","mtype_sec", "n_attend"])
# g.head()

In [4]:
mtypes_over_years = (pd.read_csv("data/ooo.csv.gz", encoding='latin-1', sep='\t')
                    .applymap(lambda x: x.split(",") if isinstance(x,str) else x)
                    )

In [5]:
mtypes_over_years.head()

,cust_id,00-17,18-24,25-29,30-34,35-39,40-44,45-49,50-54,55-59,60-64,65-69,70-74,75-84
0,11171,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[RUNION],NaN,NaN,NaN,NaN,NaN
1,11284,NaN,NaN,NaN,NaN,NaN,[HIPHOP],NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,11228,NaN,NaN,NaN,NaN,NaN,"[ALTERN, LEAGUE]",NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,11640,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[RUNION, LEAGUE]",[LEAGUE],NaN,NaN,NaN,NaN
4,11197,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[RUNION],NaN,NaN,NaN,NaN,NaN


In [6]:
v = mtypes_over_years.T

In [31]:
def _cust_features(c):
    
    f = defaultdict(int)
    
    s = v[c][v[c].notnull()]
    cid = s[0]
    
    if len(s) == 2:
        f = {"age_gr": s.index[1], "@"+s[1][0]: 1}
    elif len(s) == 3:
        f = {"age_gr": s.index[-1]}
        f.update({"@"+"->".join(tp): 1 for tp in product(s[1], s[2])})
    
    yield f

f = defaultdict(lambda: defaultdict(int))  # {cust_id: {"AFL->BALLET": 1, "TENNIS->COMEDY": 1}}

print("extracting customer features...")

t1 = time.time()

for c in range(v.shape[1]):
    for fch in _cust_features(c):
        f[v[c][0]].update(fch)
        if (c+1)%SHW == 0:
            print("extracted features for {} customers...".format(c+1))

print("done. elapsed time {:.0f} min {:.0f} sec".format(*divmod(time.time() - t1, 60)))

with open("features.pkl", "wb") as fl:
    pickle.dump(pd.DataFrame.from_dict(f, orient="index"), fl)
print("pickled feature data frame to {}".format(fl))

extracting customer features...
extracted features for 100000 customers...
extracted features for 200000 customers...
extracted features for 300000 customers...
extracted features for 400000 customers...
extracted features for 500000 customers...
extracted features for 600000 customers...
extracted features for 700000 customers...
extracted features for 800000 customers...
extracted features for 900000 customers...
extracted features for 1000000 customers...
extracted features for 1100000 customers...
extracted features for 1200000 customers...
extracted features for 1300000 customers...
extracted features for 1400000 customers...
extracted features for 1500000 customers...
extracted features for 1600000 customers...
extracted features for 1700000 customers...
extracted features for 1800000 customers...
extracted features for 1900000 customers...
extracted features for 2000000 customers...
extracted features for 2100000 customers...
extracted features for 2200000 customers...
extracted